https://www.kaggle.com/prashant111/catboost-classifier-in-python

In [ ]:
!nvidia-smi

Fri Dec  3 19:12:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/medium_train.csv /content/my_data

In [ ]:
!unzip "/content/drive/MyDrive/hackathon.zip" -d "/content/my_data"

Archive:  /content/drive/MyDrive/hackathon.zip
  inflating: /content/my_data/train.csv  
  inflating: /content/my_data/Segments.xlsx  
  inflating: /content/my_data/Problem_statement.pdf  


In [ ]:
!unzip "/content/drive/MyDrive/hackathon_test.zip" -d "/content/my_data"

Archive:  /content/drive/MyDrive/hackathon_test.zip
  inflating: /content/my_data/test.csv  


In [ ]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 76.3 MB 1.3 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import pandas as pd

In [ ]:
# train_df = pd.read_csv('/content/my_data/train.csv', parse_dates=['created'], dtype={'gamecategory': str, 'subgamecategory': str, 'bundle': str, 'shift': str, 'oblast': str, 'city': str, 'os': str, 'osv': str,} )
# test_df = pd.read_csv('/content/my_data/test.csv', parse_dates=['created'])
train_df = pd.read_csv('/content/my_data/train.csv', parse_dates=['created'])
# train_df = pd.read_csv('/content/my_data/medium_train.csv', parse_dates=['created'])

In [ ]:
test_df = pd.read_csv('/content/my_data/test.csv', parse_dates=['created'])

In [ ]:
test_df.shape[0]

11213629

In [ ]:
all_df = pd.concat([train_df.drop("Segment", axis=1), test_df])

In [ ]:
train_df = all_df[['bundle', 'gamecategory', 'subgamecategory']]

In [ ]:
train_df = train_df[train_df['bundle'].notna()]

In [ ]:
train_df = train_df[train_df['gamecategory'].notna()]

In [ ]:
train_df = train_df.drop_duplicates()

In [ ]:
train_df.head()

,bundle,gamecategory,subgamecategory
0,com.MadOut.BIG,Games,Racing
2,com.orbitalknight.ridiculousfreekick,Games,Arcade
5,com.hwg.sos,Games,Puzzle
6,com.hikergames.ArcadeHunter,Games,Arcade
7,com.lemongame.klondike.solitaire,Games,Card


In [ ]:
train_df = train_df.query("bundle == @bundles")

In [ ]:
train_df.shape[0]

39748

In [ ]:
train_df.to_csv('/content/drive/MyDrive/bundle8.csv', encoding='utf-8', sep = ';')

In [ ]:
city2 = train_df.groupby(['bundle'], as_index=False).size()
city2.head()

,bundle,size
0,1000556973,1
1,1000773093,1
2,1000876192,1
3,1001044474,1
4,1001164812,1


In [ ]:
bundles = list(city2['bundle'])

In [ ]:
city2.shape[0]

39748

In [ ]:
city2 = city2.sort_values(by=['size'], ascending=False)

In [ ]:
city2.head()

,bundle,size
27337,com.mashathedog.myvirtualpet,1
27860,com.minekopte.roblox,1
27338,com.mashinestudio.idlepoke,1
27339,com.mask.minutes3,1
27859,com.minekopte.minecoins,1


In [ ]:
city2[:987]

,bundle,size
25543,com.jockeygames.rampcar.stuntsfree.impossiblet...,4
23041,com.gf.offroad.garbage.truck.driving.parking,4
30700,com.play.io.monster.truck.demolition.derby.cra...,4
25090,com.impossible.ramp.carstuntgames,3
27025,com.ludosmobile.captain.girl.hero.ny.street.fi...,3
...,...,...
5497,1499304256,2
7905,1548712972,2
8296,1555219941,2
23602,com.grand.shooting.strike,2


In [ ]:
city2 = city2.query('size < 2')

In [ ]:
train_df.shape[0]

516667

In [ ]:
city2.to_csv('/content/drive/MyDrive/bundle3.csv', encoding='utf-8')

In [ ]:
city2 = pd.read_csv('/content/drive/MyDrive/bundle3.csv')

In [ ]:
bundles = list(city2['bundle'])

In [ ]:
print(len(bundles))

986


In [ ]:
train_df = train_df.query("bundle == @bundles")

In [ ]:
train_df.shape[0]

516667

In [ ]:
city2 = train_df.groupby(['bundle', 'gamecategory', 'subgamecategory'], as_index=False).size()
city2.head()

,bundle,gamecategory,subgamecategory,size
0,1017957701,Games,Role Playing,18
1,1017957701,Games,Simulation,42
2,1018725872,Games,Card,7
3,1018725872,Games,Strategy,3
4,1021946968,Games,Board,904


In [ ]:
city2 = city2.sort_values(by=['bundle','size'], ascending=False)

In [ ]:
city2.shape[0]

2020

In [ ]:
city2[:20]

,bundle,gamecategory,subgamecategory,size
2018,videodownloader.pinterest.video.download.pinte...,Applications,Tools,2
2019,videodownloader.pinterest.video.download.pinte...,Applications,Video Players & Editors,1
2016,solitaire.classic.cards,Games,Card,53
2017,solitaire.classic.cards,Games,Puzzle,5
2015,ru.onetwothree.wordmagic,Games,Educational,14
2014,ru.onetwothree.wordmagic,Applications,Education,5
2013,org.sabgames.paopao,Games,Puzzle,49
2012,org.sabgames.paopao,Games,Board,5
2010,om.zmmy.police.car.games.real.parking,Games,Role Playing,8
2011,om.zmmy.police.car.games.real.parking,Games,Strategy,7


In [ ]:
prev_index = -1
for index, row in city2.iterrows():
    if prev_index != -1:
        if city2.at[prev_index, 'bundle'] != city2.at[index, 'bundle']:
            city2.at[index, 'wqeq'] = 1
        else:
            city2.at[index, 'wqeq'] = 0
    else:
        city2.at[index, 'wqeq'] = 1
    prev_index = index  

In [ ]:
city2.head()

,bundle,gamecategory,subgamecategory,size,del,wqeq
2018,videodownloader.pinterest.video.download.pinte...,Applications,Tools,2,1.0,1.0
2019,videodownloader.pinterest.video.download.pinte...,Applications,Video Players & Editors,1,0.0,0.0
2016,solitaire.classic.cards,Games,Card,53,1.0,1.0
2017,solitaire.classic.cards,Games,Puzzle,5,0.0,0.0
2015,ru.onetwothree.wordmagic,Games,Educational,14,1.0,1.0


In [ ]:
qweqw = ['0.0']

In [ ]:
city3 = city2.query("wqeq != 0")

In [ ]:
city3.head()

,bundle,gamecategory,subgamecategory
2018,videodownloader.pinterest.video.download.pinte...,Applications,Tools
2016,solitaire.classic.cards,Games,Card
2015,ru.onetwothree.wordmagic,Games,Educational
2013,org.sabgames.paopao,Games,Puzzle
2010,om.zmmy.police.car.games.real.parking,Games,Role Playing


In [ ]:
city3.shape[0]

986

In [ ]:
city3 = city3.drop("size", axis=1)
city3 = city3.drop("del", axis=1)
city3 = city3.drop("wqeq", axis=1)

In [ ]:
city3.to_csv('/content/drive/MyDrive/bundle9.csv', encoding='utf-8', sep=';')

In [ ]:
city2['subgamecategory'] = city2['subgamecategory'].astype(str)

In [ ]:
qweqw = ['None']

In [ ]:
city2 = city2.query("subgamecategory != @qweqw")

In [ ]:
city2.nunique()

bundle             986
gamecategory        11
subgamecategory     44
size               316
dtype: int64

In [ ]:
city2.shape[0]

1986

In [ ]:
city2.to_csv('/content/drive/MyDrive/bundle4.csv', encoding='utf-8')

In [ ]:
city2[:20]

,bundle,gamecategory,subgamecategory,size
0,1000556973,Entertainment,None,9
1,1000773093,Games,Sports,3
2,1000876192,Games,Card,32
3,1001044474,Entertainment,None,3
4,1001164812,Games,Racing,2
5,1001259039,Education,None,9
6,1001425491,Games,Racing,160
7,1002132680,Games,Casual,1
8,1002340615,Games,Action,15
9,1002597082,Games,Family,3


In [ ]:
city2.shape[0]

40734

In [ ]:
train_df.nunique()

bundle             40734
gamecategory          27
subgamecategory       53
dtype: int64

In [ ]:
train_df.to_csv('/content/drive/MyDrive/bundle.csv', encoding='utf-8')

In [ ]:
oblast_no = ["Brestskaya Oblast’", "Gomelskaya Oblast", 'Витебская Область', 'Гродненская Область', 'Минск', 'Минская Область', 'Могилевская область']

In [ ]:
oblast_no_lower = [x.lower() for x in oblast_no]

27886087

In [ ]:
train_df.shape[0]

41774

In [ ]:
train_df.head()

,bundle,gamecategory,subgamecategory
0,com.MadOut.BIG,Games,Racing
2,com.orbitalknight.ridiculousfreekick,Games,Arcade
5,com.hwg.sos,Games,Puzzle
6,com.hikergames.ArcadeHunter,Games,Arcade
7,com.lemongame.klondike.solitaire,Games,Card


In [ ]:
train_df = train_df[:10000000]
train_df.shape[0]

10000000

In [ ]:
train_df['os'] = train_df['os'].str.lower()

In [ ]:
train_df['oblast'] = train_df['oblast'].str.lower()

In [ ]:
train_df['city'] = train_df['city'].str.lower()

In [ ]:
train_df.head()

,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,забайкальский край,чита,android,10.0
1,4,NaN,NaN,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,оренбургская область,оренбург,android,10.0.0
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,санкт-петербург,санкт-петербург,android,9.0
3,5,NaN,NaN,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,свердловская область,екатеринбург,android,9
4,4,NaN,NaN,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,московская область,звенигород,android,6.0.1


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import re

def season(month):
    if month == 12 or month == 1 or month == 2:
        return "winter"
    if 3 <= month <= 5:
        return "spring"
    if 6 <= month <= 8:
        return "summer"
    if 9 <= month <= 11:
        return "autumn"


def day_part(hour):
    if 12 <= hour < 18:
        return "day"
    if 18 <= hour <= 23:
        return "evening"
    if 6 <= hour < 12:
        return "morning"
    if 0 <= hour <= 6:
        return "night"


def real_time(shift, date):
    diff_time = str(shift[3:]) if len(str(shift)) > 3 else 0
    if type(diff_time) == type(0):
        return date
    elif diff_time[0:1] == "+":
        return date + timedelta(hours=int(diff_time[1:]))
    elif diff_time[0:1] == "-":
        return date - timedelta(hours=int(diff_time[1:]))
    return date

def city_type(population):
    if population < 10000:
        return 'village'
    elif population < 100000:
        return 'small'
    elif population < 500000:
        return 'medium'
    elif population < 1000000:
        return 'big'
    elif population >= 1000000:
        return 'million'
    return 'small'


In [ ]:
df = pd.read_csv('/content/my_data/train.csv', parse_dates=['created'])
df = df[36000000:]
print(df.shape[0])
os = pd.read_csv('/content/drive/MyDrive/os.csv', sep=',')
# city = pd.read_csv('/content/drive/MyDrive/city.csv', sep=';', encoding="utf-8")
cat = pd.read_csv('/content/drive/MyDrive/cat.csv', sep=';')
bundle8 = pd.read_csv('/content/drive/MyDrive/bundle11.csv', sep=';')
# city.drop('id', 1, inplace=True)

df['os'] = df['os'].str.lower()
# city['oblast'] = city['oblast'].str.strip().str.lower()
# city['city'] = city['city'].str.strip().str.lower()
# df['city'] = df['city'].str.strip().str.lower()
# df['oblast'] = df['oblast'].str.strip().str.lower()
# df['os'] = df['os'].str.lower()

# city['oblast'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# city['city'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# df['city'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# df['oblast'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)


df["created"] = pd.to_datetime(df["created"], format='%Y-%m-%d %H:%M:%S')
df['diff_time'] = df.apply(lambda row: str(row['shift'])[3:] if len(str(row['shift'])) > 3 else 0, axis=1)
df['real_time'] = df.apply(lambda row: real_time(row['shift'], row["created"]), axis=1)
df['weekday'] = df.apply(lambda row: row['real_time'].weekday(), axis=1)
df['season'] = df.apply(lambda row: season(row['real_time'].month), axis=1)
df['day_part'] = df.apply(lambda row: day_part(row['real_time'].hour), axis=1)

# mer.to_csv('mer.csv', sep=';', encoding='utf-8')

8854516


In [ ]:
# mer = df.merge(city, on=['oblast', 'city'], how='left')
mer = df.merge(os, on=['os', 'osv'], how='left')
mer = mer.merge(bundle8, on=['bundle'], how='left')
mer = mer.merge(cat, on=['gamecategory_y', 'subgamecategory_y'], how='left')
print(mer.shape[0])

8854516


In [ ]:
# mer['city_type'] = mer.apply(lambda row: city_type(row['population']), axis=1)

In [ ]:
mer = mer.drop("gamecategory_x", axis=1)
mer = mer.drop("subgamecategory_x", axis=1)
mer = mer.drop("gamecategory_y", axis=1)
mer = mer.drop("subgamecategory_y", axis=1)
mer = mer.drop("Unnamed: 0", axis=1)
mer = mer.drop("diff_time", axis=1)
mer = mer.drop("shift", axis=1)
# mer = mer.drop("type", axis=1)
# mer = mer.drop("children", axis=1)
# mer = mer.drop("population", axis=1)
mer = mer.drop("created", axis=1)

In [ ]:
mer.head()

,Segment,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,4,com.psv.ladybug.color_by_number,Москва,Москва,android,10.0,2021-07-06 21:48:19,1,summer,evening,10,Games,Board
1,4,paint.by.number.pixel.art.coloring.drawing.puzzle,Москва,Москва,android,9.0,2021-07-05 20:33:36,0,summer,evening,9,Games,Board
2,5,com.loltap.farmland,Удмуртия,Ижевск,android,9.0,2021-08-02 16:47:30,0,summer,day,9,Games,Adventure
3,4,bubble.shoot.fruit.splash.game2,Иркутская область,Иркутск,android,9.0,2021-07-10 23:22:07,5,summer,evening,9,Games,Casual
4,5,com.hazmob.blockstrikefps,NaN,NaN,android,9.0,2021-09-15 13:44:19,2,autumn,day,9,Games,Action


In [ ]:
print(mer.shape[0])

8854516


In [ ]:
mer.to_csv('/content/drive/MyDrive/mer_train_fin4.csv', sep=';', encoding='utf-8')

In [ ]:
train_df.shape[0]

9999949

In [ ]:
train_df = train_df.query("oblast != @oblast_no_lower")

In [ ]:
train_df['oblast2'] = train_df['oblast']

In [ ]:
train_df['city'][0] = str(train_df['city2'][0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_df.head()

,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,city2
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10.0,Чита
1,4,NaN,NaN,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,Android,10.0.0,Оренбург
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9.0,Санкт-Петербург
3,5,NaN,NaN,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9,Екатеринбург
4,4,NaN,NaN,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6.0.1,Звенигород


In [ ]:
train_df.assign(city2=lambda x: x.city)

In [ ]:
city2 = train_df.groupby('os').size()
city2.head()

os
android    8804767
ios        1195139
dtype: int64

In [ ]:
city2 = train_df.groupby(['oblast', 'city']).size()
city2.head()

In [ ]:
city2 = train_df.groupby(['oblast', 'city']).size()
city2.head()

oblast              city      
Brestskaya Oblast’  Berëza         1
                    Барановичи     6
                    Брест         21
                    Иваново        1
                    Кобрин         6
dtype: int64

In [ ]:
city2.shape[0]

2623

In [ ]:
city2.to_csv('/content/drive/MyDrive/osv.csv', encoding='utf-8')

In [ ]:
city = pd.DataFrame(train_df[['city', 'oblast', 'city2']]).groupby('city', as_index=False).aggregate({'city2': 'count'})
city.head()

,city,city2
0,Berëza,1
1,Gomel’,24
2,Oktyabr’skiy,13
3,Абадзехская,56
4,Абаза,1253


In [ ]:
city = pd.DataFrame(train_df[['oblast', 'oblast2']]).groupby('oblast', as_index=False).aggregate({'oblast2': 'count'})
city.head()

In [ ]:
city.shape[0]

2498

In [ ]:
city.to_csv('/content/drive/MyDrive/city.csv', encoding='utf-8')

In [ ]:
train_df.shape[0]

44854516

In [ ]:
train_df = train_df[:4854516]
train_df.shape[0]

4854516

In [ ]:
mer['category'] = mer['category'].astype(str)
mer['subcategory'] = mer['subcategory'].astype(str)
mer['bundle'] = mer['bundle'].astype(str)
mer['season'] = mer['season'].astype(str)
mer['oblast'] = mer['oblast'].astype(str)
mer['city'] = mer['city'].astype(str)
mer['os'] = mer['os'].astype(str)
mer['osv'] = mer['osv'].astype(str)
mer['day_part'] = mer['day_part'].astype(str)
# mer['city_type'] = mer['city_type'].astype(str)
mer['os_global'] = mer['os_global'].astype(str)

In [ ]:
mer.head()

,Segment,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,4,com.MadOut.BIG,Забайкальский Край,Чита,android,10.0,2021-07-06 00:07:40,1,summer,night,10,Games,Racing
1,4,com.easybrain.solitaire.klondike.free,Оренбургская область,Оренбург,android,10.0.0,2021-07-10 12:38:42,5,summer,day,10,Games,Card
2,5,com.orbitalknight.ridiculousfreekick,Санкт-Петербург,Санкт-Петербург,android,9.0,2021-08-04 13:34:29,2,summer,day,9,Games,Arcade
3,5,tcouchgind.scooterextreme.scooter,Свердловская область,Екатеринбург,android,9,2021-08-06 09:35:27,4,summer,morning,9,nan,nan
4,4,com.FidgetTrading3D.game,Московская область,Звенигород,android,6.0.1,2021-08-02 20:43:59,0,summer,evening,6,Games,Casual


In [ ]:
train_df.head()

,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10.0
1,4,nan,nan,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,Android,10.0.0
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9.0
3,5,nan,nan,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9
4,4,nan,nan,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6.0.1


In [ ]:
test_df.head()

,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0


In [ ]:
mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4854516 entries, 0 to 4854515
Data columns (total 13 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Segment      int64         
 1   bundle       object        
 2   oblast       object        
 3   city         object        
 4   os           object        
 5   osv          object        
 6   real_time    datetime64[ns]
 7   weekday      int64         
 8   season       object        
 9   day_part     object        
 10  os_global    object        
 11  category     object        
 12  subcategory  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 518.5+ MB


In [ ]:
train_df.nunique()

Segment                  5
gamecategory            28
subgamecategory         53
bundle               42456
created            1405261
shift                   12
oblast                  91
city                  2433
os                       5
osv                    217
dtype: int64

In [ ]:
mer = pd.read_csv('/content/drive/MyDrive/mer_train_fin2.csv', sep=';', parse_dates=['real_time'])

In [ ]:
mer.head()

,Segment,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,3,com.zeptolab.cats.google,Москва,Москва,android,11.0,2021-09-20 09:54:55,0,autumn,morning,11.0,Games,Action
1,2,com.hgamesart.crimedriver,Сахалин,Южно-Сахалинск,android,9.0,2021-08-03 09:01:17,1,summer,morning,9.0,Games,Action
2,3,com.pronetis.ironball2,Воронежская область,Воронеж,android,10.0.0,2021-08-03 21:38:21,1,summer,evening,10.0,Games,Casual
3,3,com.bigcake.android.bpdaily,Саратовская область,Саратов,android,7.0,2021-07-08 14:08:37,3,summer,day,7.0,Games,Puzzle
4,3,com.outfit7.mytalkingangela2,Хабаровский край,Хабаровск,android,9,2021-08-03 22:42:52,1,summer,evening,9.0,Games,Casual


In [ ]:
mer = mer.drop("Unnamed: 0", axis=1)

In [ ]:
mer.shape[0]

10046694

In [ ]:
mer = mer[mer['bundle'].notna()]

In [ ]:
mer = mer[mer['city'].notna()]

In [ ]:
mer['category'] = mer['category'].astype(str)
mer['subcategory'] = mer['subcategory'].astype(str)
mer['bundle'] = mer['bundle'].astype(str)
mer['season'] = mer['season'].astype(str)
mer['oblast'] = mer['oblast'].astype(str)
mer['city'] = mer['city'].astype(str)
mer['os'] = mer['os'].astype(str)
mer['osv'] = mer['osv'].astype(str)
mer['day_part'] = mer['day_part'].astype(str)
# mer['city_type'] = mer['city_type'].astype(str)
mer['os_global'] = mer['os_global'].astype(str)

In [ ]:
# mer['Segment'] = mer.apply(lambda row: row['Segment'] - 1, axis=1)

In [ ]:
X = mer.drop("Segment", axis=1)
y = mer["Segment"]

In [ ]:
mer.to_csv('/content/drive/MyDrive/prom_itog.csv', encoding='utf-8')

In [ ]:
mer.shape[0]

4854516

In [ ]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [ ]:
cat_features = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]
print(cat_features)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1500, 
    # learning_rate=0.1, 
    task_type="GPU",
    devices='0',
    loss_function='MultiClass'
    # custom_loss=['AUC', 'Accuracy']
    #loss_function='CrossEntropy'
)

In [ ]:
model.fit(
    X, y,
    cat_features=cat_features,
    # eval_set=(X_val, y_val),
)

Learning rate set to 0.303398
0:	learn: 1.2957900	total: 556ms	remaining: 13m 52s
1:	learn: 1.1501750	total: 1.15s	remaining: 14m 19s
2:	learn: 1.0704687	total: 1.7s	remaining: 14m 9s
3:	learn: 1.0208097	total: 2.18s	remaining: 13m 36s
4:	learn: 0.9860585	total: 2.7s	remaining: 13m 28s
5:	learn: 0.9452867	total: 3.2s	remaining: 13m 16s
6:	learn: 0.9190560	total: 3.78s	remaining: 13m 26s
7:	learn: 0.9026577	total: 4.31s	remaining: 13m 24s
8:	learn: 0.8874453	total: 4.87s	remaining: 13m 26s
9:	learn: 0.8761414	total: 5.42s	remaining: 13m 27s
10:	learn: 0.8676008	total: 5.91s	remaining: 13m 19s
11:	learn: 0.8617845	total: 6.41s	remaining: 13m 15s
12:	learn: 0.8565328	total: 6.84s	remaining: 13m 2s
13:	learn: 0.8528902	total: 7.27s	remaining: 12m 51s
14:	learn: 0.8469971	total: 7.79s	remaining: 12m 50s
15:	learn: 0.8415341	total: 8.28s	remaining: 12m 48s
16:	learn: 0.8366023	total: 8.83s	remaining: 12m 50s
17:	learn: 0.8334229	total: 9.34s	remaining: 12m 49s
18:	learn: 0.8306681	total: 9.7

In [ ]:
model.fit(
    X, y,
    cat_features=cat_features,
    init_model=model
    # eval_set=(X_val, y_val),
)
# model.fit(train_data, train_labels, init_model=model1)

Learning rate set to 0.5


CatBoostError: ignored

In [ ]:
model2 = CatBoostClassifier(
    iterations=1000, 
    # learning_rate=0.1, 
    task_type="GPU",
    devices='0',
    # custom_loss=['AUC', 'Accuracy']
    #loss_function='CrossEntropy'
)

In [ ]:
model2.fit(
    X, y,
    cat_features=cat_features,
    init_model=model
    # eval_set=(X_val, y_val),
)

Learning rate set to 0.366655


CatBoostError: ignored

In [ ]:
importances = model.feature_importances_
feature_list = list(X.columns)
feature_results = pd.DataFrame({'feature': feature_list,'importance': importances})
feature_results = feature_results.sort_values('importance',ascending = False).reset_index(drop=True)
feature_results.head(10)

,feature,importance
0,real_time,51.658311
1,day_part,11.246987
2,bundle,7.156988
3,osv,6.579672
4,season,5.600533
5,weekday,5.501862
6,oblast,4.181142
7,os,2.701559
8,subcategory,2.027133
9,city,1.483719


In [ ]:
importances = model.feature_importances_
feature_list = list(X.columns)
feature_results = pd.DataFrame({'feature': feature_list,'importance': importances})
feature_results = feature_results.sort_values('importance',ascending = False).reset_index(drop=True)
feature_results.head(15)

,feature,importance
0,real_time,50.274165
1,day_part,11.002975
2,bundle,7.205385
3,oblast,6.804980
4,osv,6.541171
5,weekday,6.355507
6,season,4.799131
7,os,2.296069
8,subcategory,2.049655
9,city,1.409587


In [ ]:
model.save_model("model3")

# from_file = CatBoostClassifier()

# from_file.load_model("model")

In [ ]:
print(model.predict(data=X[:20]))

[[4]
 [4]
 [5]
 [5]
 [4]
 [3]
 [4]
 [5]
 [3]
 [5]
 [3]
 [5]
 [3]
 [5]
 [5]
 [5]
 [5]
 [5]
 [4]
 [3]]


In [ ]:
train_df.iloc[:20]['Segment']

4

In [ ]:
train_df[:20]

,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10.0
1,4,nan,nan,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,Android,10.0.0
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9.0
3,5,nan,nan,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9
4,4,nan,nan,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6.0.1
5,5,Games,Puzzle,com.hwg.sos,2021-09-18 09:44:24,MSK,Татарстан,Альметьевск,android,7.1
6,4,Games,Arcade,com.hikergames.ArcadeHunter,2021-08-02 08:52:24,MSK,Москва,Москва,android,11.0
7,5,Games,Card,com.lemongame.klondike.solitaire,2021-09-18 02:36:11,MSK,Санкт-Петербург,Санкт-Петербург,android,8.1.0
8,3,nan,nan,com.beingame.zc.zombie.shelter.survival,2021-08-07 15:29:59,MSK,Санкт-Петербург,Санкт-Петербург,android,11.0.0
9,5,nan,nan,com.phonemaster.jewelhunter,2021-08-03 15:45:45,MSK+3,Новосибирская область,Новосибирск,android,9


In [ ]:
from catboost import CatBoostClassifier, Pool
model = CatBoostClassifier()

model.load_model("model2")

In [ ]:
n = 10046000
count_true = 0
pred = model.predict(data=X[:n])
y_true = mer.iloc[:n]['Segment']
count = 0
for i in range(n):
    try:
        count += 1
        if pred[i][0] == y_true[i]:
          count_true += 1
    except:
        continue
print(count_true)
print(count_true / count)

1264381
0.12585914791956998


In [ ]:
y[10]

4

In [ ]:
n = 4854516
count_true = 0
pred = model2.predict(data=X[:n])
y_true = train_df.iloc[:n]['Segment']

for i in range(n):
    if pred[i][0] == y_true[i]:
      count_true += 1
print(count_true)
print(count_true / n)

3432221
0.7070161062400454


In [ ]:
test_df['os'] = test_df['os'].str.lower()
# city['oblast'] = city['oblast'].str.strip().str.lower()
# city['city'] = city['city'].str.strip().str.lower()
# df['city'] = df['city'].str.strip().str.lower()
# df['oblast'] = df['oblast'].str.strip().str.lower()
# df['os'] = df['os'].str.lower()

# city['oblast'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# city['city'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# df['city'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)
# df['oblast'] = re.sub("[^a-z0-9]+","", str(city['oblast']), flags=re.IGNORECASE)


test_df["created"] = pd.to_datetime(test_df["created"], format='%Y-%m-%d %H:%M:%S')
test_df['diff_time'] = test_df.apply(lambda row: str(row['shift'])[3:] if len(str(row['shift'])) > 3 else 0, axis=1)
test_df['real_time'] = test_df.apply(lambda row: real_time(row['shift'], row["created"]), axis=1)
test_df['weekday'] = test_df.apply(lambda row: row['real_time'].weekday(), axis=1)
test_df['season'] = test_df.apply(lambda row: season(row['real_time'].month), axis=1)
test_df['day_part'] = test_df.apply(lambda row: day_part(row['real_time'].hour), axis=1)

In [ ]:
os = pd.read_csv('/content/drive/MyDrive/os.csv', sep=',')
# city = pd.read_csv('/content/drive/MyDrive/city.csv', sep=';', encoding="utf-8")
cat = pd.read_csv('/content/drive/MyDrive/cat.csv', sep=';')
bundle8 = pd.read_csv('/content/drive/MyDrive/bundle11.csv', sep=';')

In [ ]:
mer2 = test_df.merge(os, on=['os', 'osv'], how='left')
mer2 = mer2.merge(bundle8, on=['bundle'], how='left')
mer2 = mer2.merge(cat, on=['gamecategory_y', 'subgamecategory_y'], how='left')
print(mer2.shape[0])

11213629


In [ ]:
mer2 = mer2.drop("gamecategory_x", axis=1)
mer2 = mer2.drop("subgamecategory_x", axis=1)
mer2 = mer2.drop("gamecategory_y", axis=1)
mer2 = mer2.drop("subgamecategory_y", axis=1)
mer2 = mer2.drop("Unnamed: 0", axis=1)
mer2 = mer2.drop("diff_time", axis=1)
mer2 = mer2.drop("shift", axis=1)
# mer = mer.drop("type", axis=1)
# mer = mer.drop("children", axis=1)
# mer = mer.drop("population", axis=1)
mer2 = mer2.drop("created", axis=1)

In [ ]:
print(mer2.shape[0])

11213629


In [ ]:
mer2.to_csv('/content/drive/MyDrive/mer_test_fin.csv', encoding='utf-8')

In [ ]:
mer2['category'] = mer2['category'].astype(str)
mer2['subcategory'] = mer2['subcategory'].astype(str)
mer2['bundle'] = mer2['bundle'].astype(str)
mer2['season'] = mer2['season'].astype(str)
mer2['oblast'] = mer2['oblast'].astype(str)
mer2['city'] = mer2['city'].astype(str)
mer2['os'] = mer2['os'].astype(str)
mer2['osv'] = mer2['osv'].astype(str)
mer2['day_part'] = mer2['day_part'].astype(str)
# mer['city_type'] = mer['city_type'].astype(str)
mer2['os_global'] = mer2['os_global'].astype(str)

In [ ]:
mer2.head()

,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,com.allgoritm.youla,Свердловская область,Екатеринбург,android,10.0,2021-09-19 19:31:33,6,autumn,evening,10,Shopping,Shopping
1,com.ChocochocoStd.RapBattle,Ямало-Ненецкий АО,Новый Уренгой,android,10.0,2021-08-03 19:13:17,1,summer,evening,10,nan,nan
2,1387897651,Москва,Москва,ios,14.4.0,2021-09-17 15:54:00,4,autumn,day,14,Games,Board
3,com.cooking.family.diary.fever.food.city.craze...,Краснодарский край,Краснодар,android,9.0.0,2021-07-05 23:34:59,0,summer,evening,9,Games,Simulation
4,com.pedometer.stepcounter.tracker,Татарстан,Набережные Челны,android,10.0,2021-07-08 15:15:21,3,summer,day,10,Health & Fitness,Health & Fitness


In [ ]:
from catboost import CatBoostClassifier, Pool
from_file = CatBoostClassifier()

from_file.load_model("model2")

In [ ]:
pred_test = from_file.predict(data=mer2)

In [ ]:
pred_test[0][0]

3

In [ ]:
n = 11213629
res = [0, 0, 0, 0, 0, 0]
for num, i in enumerate(pred_test):
    res[i[0]] += 1



TypeError: ignored

In [ ]:
for i in res:
    print(i*100/n)

0.0
0.1553110059196715
2.212102790274228
29.369939026875244
20.782647615682666
47.47999956124819


In [ ]:
mer2.to_csv('/content/drive/MyDrive/mer_test.csv', encoding='utf-8')

# $$CatBoost\ Tutorial$$

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/python_tutorial.ipynb)

In this tutorial we would explore some base cases of using catboost, such as model training, cross-validation and predicting, as well as some useful features like early stopping,  snapshot support, feature importances and parameters tuning.
  
You could run this tutorial in Google Colaboratory environment with free CPU or GPU. Just click on this <a href="https://colab.research.google.com/github/catboost/tutorials/blob/master/python_tutorial.ipynb" target="_blank" title="Colab">link</a>.

## $$Contents$$
* [1. Data Preparation](#$$1.\-Data\-Preparation$$)
    * [1.1 Data Loading](#1.1-Data-Loading)
    * [1.2 Feature Preparation](#1.2-Feature-Preparation)
    * [1.3 Data Splitting](#1.3-Data-Splitting)
* [2. CatBoost Basics](#$$2.\-CatBoost\-Basics$$)
    * [2.1 Model Training](#2.1-Model-Training)
    * [2.2 Model Cross-Validation](#2.2-Model-Cross-Validation)
    * [2.3 Model Applying](#2.3-Model-Applying)
* [3. CatBoost Features](#$$3.\-CatBoost\-Features$$)
    * [3.1 Using the best model](#3.1-Using-the-best-model)
    * [3.2 Early Stopping](#3.2-Early-Stopping)
    * [3.3 Using Baseline](#3.3-Using-Baseline)
    * [3.4 Snapshot Support](#3.4-Snapshot-Support)
    * [3.5 User Defined Objective Function](#3.5-User-Defined-Objective-Function)
    * [3.6 User Defined Metric Function](#3.6-User-Defined-Metric-Function)
    * [3.7 Staged Predict](#3.7-Staged-Predict)
    * [3.8 Feature Importances](#3.8-Feature-Importances)
    * [3.9 Eval Metrics](#3.9-Eval-Metrics)
    * [3.10 Learning Processes Comparison](#3.10-Learning-Processes-Comparison)
    * [3.11 Model Saving](#3.11-Model-Saving)
* [4. Parameters Tuning](#$$4.\-Parameters\-Tuning$$)

## $$1.\ Data\ Preparation$$
### 1.1 CatBoost installation
If you have not already installed CatBoost, you can do so by running '!pip install catboost' command.  
  
Also you should install ipywidgets package and run special command before launching jupyter notebook to draw plots.

In [ ]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

### 1.2 Data Loading
The data for this tutorial can be obtained from [this page](https://www.kaggle.com/c/titanic/data) (you would have to register a kaggle account or just login with facebook or google+) or you could use catboost.datasets as in code below.

In [ ]:
from catboost.datasets import titanic
import numpy as np

train_df, test_df = titanic()

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 1.3 Feature Preparation
First of all let's check how many absent values do we have:

In [ ]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

As we can see, **`Age`**, **`Cabin`** and **`Embarked`** indeed have some missing values, so let's fill them with some number way out of their distributions - so the model would be able to easily distinguish between them and take it into account:

In [ ]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Now let's separate features and label variable:

In [ ]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

Pay attention that our features are of different types - some of them are numeric, some are categorical, and some are even just strings, which normally should be handled in some specific way (for example encoded with bag-of-words representation). But in our case we could treat these string features just as categorical one - all the heavy lifting is done inside CatBoost. How cool is that? :)

In [ ]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


### 1.4 Data Splitting
Let's split the train data into training and validation sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

## $$2.\ CatBoost\ Basics$$

Let's make necessary imports.

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

### 2.1 Model Training
Now let's create the model itself. We will go here with default parameters, as they provide a _really_ good baseline almost all the time. The only thing we would like to specify here is `custom_loss` parameter, as this would give us an ability to see what's going on in terms of this competition metric - accuracy, as well as to be able to watch for logloss, as it would be more smooth on dataset of such size.

In [ ]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [ ]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

As you can see, it is possible to watch our model learn through verbose output or with nice plots (personally I would definately go with the second option - just check out those plots: you can, for example, zoom in areas of interest!)

With this we can see that the best accuracy value of **0.8296** (on validation set) was acheived on **150** boosting step.

### 2.2 Model Cross-Validation

It is good to validate your model, but to cross-validate it - even better. And also with plots! So with no more words:

In [ ]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Now we have values of our loss functions at each boosting step averaged by 3 folds, which should provide us with a more accurate estimation of our model performance:

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.83±0.02 on step 355


In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8294051627384961


As we can see, our initial estimation of performance on single validation fold was too optimistic - that is why cross-validation is so important!

### 2.3 Model Applying
All you have to do to get predictions is

In [ ]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 0 0 0 1 0 1 0 1 0]
[[0.85473931 0.14526069]
 [0.76313031 0.23686969]
 [0.88972889 0.11027111]
 [0.87876173 0.12123827]
 [0.3611047  0.6388953 ]
 [0.90513381 0.09486619]
 [0.33434185 0.66565815]
 [0.78468564 0.21531436]
 [0.39429048 0.60570952]
 [0.94047549 0.05952451]]


But let's try to get a better predictions and Catboost features help us in it.

## $$3.\ CatBoost\ Features$$
You may have noticed that on model creation step I've specified not only `custom_loss` but also `random_seed` parameter. That was done in order to make this notebook reproducible - by default catboost chooses some random value for seed:

In [ ]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 0


Let's define some params and create `Pool` for more convenience. It stores all information about dataset (features, labeles, categorical features indices, weights and and much more).

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

### 3.1 Using the best model
If you essentially have a validation set, it's always better to use the `use_best_model` parameter during training. By default, this parameter is enabled. If it is enabled, the resulting trees ensemble is shrinking to the best iteration.

In [ ]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.7982

Best model validation accuracy: 0.8251


### 3.2 Early Stopping
If you essentially have a validation set, it's always easier and better to use early stopping. This feature is similar to the previous one, but only in addition to improving the quality it still saves time.

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 9.42 s, sys: 368 ms, total: 9.79 s
Wall time: 1.1 s


In [ ]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 1.52 s, sys: 108 ms, total: 1.62 s
Wall time: 195 ms


In [ ]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Simple model tree count: 500
Simple model validation accuracy: 0.7982

Early-stopped model tree count: 82
Early-stopped model validation accuracy: 0.8072


So we get better quality in a shorter time.

Though as was shown earlier simple validation scheme does not precisely describes model out-of-train score (may be biased because of dataset split) it is still nice to track model improvement dynamics - and thereby as we can see from this example it is really good to stop boosting process earlier (before the overfitting kicks in)

### 3.3 Using Baseline
It is posible to use pre-training results (baseline) for training.

In [ ]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

### 3.4 Snapshot Support
Catboost supports snapshots. You can use it for recovering training after an interruption or for starting training with previous results. 

In [ ]:
params_with_snapshot = params.copy()
params_with_snapshot.update({
    'iterations': 5,
    'learning_rate': 0.5,
    'logging_level': 'Verbose'
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)
params_with_snapshot.update({
    'iterations': 10,
    'learning_rate': 0.1,
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)

0:	learn: 0.8053892	test: 0.7937220	best: 0.7937220 (0)	total: 1.39ms	remaining: 5.54ms
1:	learn: 0.8008982	test: 0.7982063	best: 0.7982063 (1)	total: 3ms	remaining: 4.5ms
2:	learn: 0.8008982	test: 0.7937220	best: 0.7982063 (1)	total: 4.13ms	remaining: 2.75ms
3:	learn: 0.8113772	test: 0.7892377	best: 0.7982063 (1)	total: 5.52ms	remaining: 1.38ms
4:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 6.67ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4

5:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 8.14ms	remaining: 5.89ms
6:	learn: 0.8248503	test: 0.8026906	best: 0.8026906 (4)	total: 9.64ms	remaining: 4.46ms
7:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 10.8ms	remaining: 2.76ms
8:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 11.4ms	remaining: 1.19ms
9:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 12.7ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4



### 3.5 User Defined Objective Function
It is possible to create your own objective function. Let's create logloss objective function.

In [ ]:
# for performance reasons it is better to install `numba` package for working with user defined functions
!pip install numba

In [ ]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers which have only __len__ and __getitem__ defined).
        # weights parameter can be None.
        #
        # To understand what these parameters mean, assume that there is
        # a subset of your dataset that is currently being processed.
        # approxes contains current predictions for this subset,
        # targets contains target values you provided with the dataset.
        #
        # This function should return a list of pairs (der1, der2), where
        # der1 is the first derivative of the loss function with respect
        # to the predicted value, and der2 is the second derivative.
        #
        # In our case, logloss is defined by the following formula:
        # target * log(sigmoid(approx)) + (1 - target) * (1 - sigmoid(approx))
        # where sigmoid(x) = 1 / (1 + e^(-x)).
        
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42, 
    loss_function=LoglossObjective(), 
    eval_metric=metrics.Logloss()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

0:	learn: 0.6827074	total: 16.7ms	remaining: 150ms
1:	learn: 0.6723302	total: 30.8ms	remaining: 123ms
2:	learn: 0.6619449	total: 43.3ms	remaining: 101ms
3:	learn: 0.6521466	total: 60.3ms	remaining: 90.5ms
4:	learn: 0.6435227	total: 73.4ms	remaining: 73.4ms
5:	learn: 0.6353848	total: 87.8ms	remaining: 58.5ms
6:	learn: 0.6277210	total: 99.6ms	remaining: 42.7ms
7:	learn: 0.6210282	total: 111ms	remaining: 27.8ms
8:	learn: 0.6141958	total: 123ms	remaining: 13.6ms
9:	learn: 0.6073236	total: 135ms	remaining: 0us


### 3.6 User Defined Metric Function
Also it is possible to create your own metric function. Let's create logloss metric function.

In [ ]:
class LoglossMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        # approxes is a list of indexed containers
        # (containers with only __len__ and __getitem__ defined),
        # one container per approx dimension.
        # Each container contains floats.
        # weight is a one dimensional indexed container.
        # target is float.
        
        # weight parameter can be None.
        # Returns pair (error, weights sum)
        
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += -w * (target[i] * approx[i] - np.log(1 + np.exp(approx[i])))

        return error_sum, weight_sum

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42, 
    loss_function=metrics.Logloss(),
    eval_metric=LoglossMetric()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 0.5521578	total: 6.34ms	remaining: 57.1ms
1:	learn: 0.4885686	total: 11.9ms	remaining: 47.5ms
2:	learn: 0.4607664	total: 17.4ms	remaining: 40.5ms
3:	learn: 0.4418819	total: 22.6ms	remaining: 33.9ms
4:	learn: 0.4278162	total: 28.4ms	remaining: 28.4ms
5:	learn: 0.4151036	total: 35.1ms	remaining: 23.4ms
6:	learn: 0.4099336	total: 40.6ms	remaining: 17.4ms
7:	learn: 0.4095363	total: 45.8ms	remaining: 11.5ms
8:	learn: 0.4032867	total: 51.3ms	remaining: 5.7ms
9:	learn: 0.3929586	total: 56.6ms	remaining: 0us


### 3.7 Staged Predict
CatBoost model has `staged_predict` method. It allows you to iteratively get predictions for a given range of trees.

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
ntree_start, ntree_end, eval_period = 3, 9, 2
predictions_iterator = model.staged_predict(validate_pool, 'Probability', ntree_start, ntree_end, eval_period)
for preds, tree_count in zip(predictions_iterator, range(ntree_start, ntree_end, eval_period)):
    print('First class probabilities using the first {} trees: {}'.format(tree_count, preds[:5, 1]))

First class probabilities using the first 3 trees: [0.53597869 0.41039128 0.42057479 0.64281031 0.46576685]
First class probabilities using the first 5 trees: [0.63722688 0.42492029 0.46209302 0.70926021 0.44280772]
First class probabilities using the first 7 trees: [0.66964764 0.42409144 0.46124982 0.76101033 0.47205986]


### 3.8 Feature Importances
Sometimes it is very important to understand which feature made the greatest contribution to the final result. To do this, the CatBoost model has a `get_feature_importance` method.

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 59.0040920142686
Pclass: 16.340887169747038
Ticket: 6.028107169932206
Cabin: 3.8347242202560192
Fare: 3.712969667934385
Age: 3.4844512041824824
Parch: 3.378089740355865
Embarked: 2.313999407289956
SibSp: 1.902679406033451
PassengerId: 0.0
Name: 0.0


This shows that features **`Sex`** and **`Pclass`** had the biggest influence on the result.

### 3.9 Eval Metrics
The CatBoost has a `eval_metrics` method that allows to calculate a given metrics on a given dataset. And to draw them of course:)

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
eval_metrics = model.eval_metrics(validate_pool, [metrics.AUC()], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
print(eval_metrics['AUC'][:6])

[0.8627368774106994, 0.8623176253563642, 0.8602213650846889, 0.8514170719436525, 0.8495723629045783, 0.8569092738554419]


### 3.10 Learning Processes Comparison
You can also compare different models learning process on a single plot.

In [ ]:
model1 = CatBoostClassifier(iterations=100, depth=1, train_dir='model_depth_1/', logging_level='Silent')
model1.fit(train_pool, eval_set=validate_pool)
model2 = CatBoostClassifier(iterations=100, depth=5, train_dir='model_depth_5/', logging_level='Silent')
model2.fit(train_pool, eval_set=validate_pool);

In [ ]:
from catboost import MetricVisualizer
widget = MetricVisualizer(['model_depth_1', 'model_depth_5'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 3.11 Model Saving
It is always really handy to be able to dump your model to disk (especially if training took some time).

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump');

# $$4.\ Parameters\ Tuning$$
While you could always select optimal number of iterations (boosting steps) by cross-validation and learning curve plots, it is also important to play with some of model parameters, and we would like to pay some special attention to `l2_leaf_reg` and `learning_rate`.

In this section, we'll select these parameters using the **`hyperopt`** package.

In [ ]:
!pip install hyperopt

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric=metrics.Accuracy(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
    )
    
    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params(),
        logging_level='Silent',
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:59<00:00,  4.79s/trial, best loss: 0.16386083052749711]
{'l2_leaf_reg': 1.0, 'learning_rate': 0.0450866712211308}


Now let's get all cv data with best parameters:

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric=metrics.Accuracy(),
    random_seed=42,
    verbose=False,
    loss_function=metrics.Logloss(),
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

Training on fold [0/3]

bestTest = 0.8417508418
bestIteration = 262

Training on fold [1/3]

bestTest = 0.8451178451
bestIteration = 269

Training on fold [2/3]

bestTest = 0.8215488215
bestIteration = 284



In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8361391694725029


Recall that with default parameters out cv score was 0.8283, and thereby we have (probably not statistically significant) some improvement.

### Make submission
Now we would re-train our tuned model on all train data that we have

In [ ]:
model.fit(X, y, cat_features=categorical_features_indices)

And finally let's prepare the submission file:

In [ ]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [ ]:
submisstion.to_csv('submission.csv', index=False)

Finally you can make submission at [Titanic Kaggle competition](https://www.kaggle.com/c/titanic).

That's it! Now you can play around with CatBoost and win some competitions! :)